# Clase 7: Análisis de Entrada I: herramientas adicionales

Prof. Tito Homem-de-Mello

## Trabajaremos con el archivo "energy_dataset.csv"

In [ ]:
install.packages('fitdistrplus')
install.packages('ggplot2')
install.packages('nortest')
install.packages('tidyr')
install.packages('data.table')

In [ ]:
library(fitdistrplus)
library(data.table)
library(nortest)
library(ggplot2)
library(tidyr)

In [ ]:
#Ajustes para gráficos via ggplot
mytheme <- theme(plot.title = element_text(size = 22),
                    axis.title.y=element_text(size = 20),
                    axis.text.y=element_text(size = 15),
                    axis.title.x=element_text(size = 20),
                    axis.text.x=element_text(size = 15),
                    legend.position="bottom",
                    legend.text=element_text(size = 15),
                    legend.title=element_text(size = 15))

In [ ]:
energy <- fread("energy_dataset.csv")
head(energy)

## Queremos simular el precio real ("price actual"). ¡Hay que definir una distribución!

In [ ]:
precio <- energy$`price actual`
hist(precio)
cat(length(precio), "datos")

### Bondad de ajuste - plots

In [ ]:
a_norm <- fitdist(precio, "norm")
a_logis <- fitdist(precio, "logis")

a_norm
a_logis
plot.legend <- c("Normal","Logistic")
denscomp(list(a_norm,a_logis), fitcol=c(2,4), fitlty=1, legendtext=plot.legend)


In [ ]:
ppcomp(list(a_norm,a_logis),fitcol=c(2,4),legendtext=plot.legend)

In [ ]:
qqcomp(list(a_norm,a_logis),fitcol=c(2,4),legendtext=plot.legend)

### Bondad de ajuste - tests estadísticos

In [ ]:
gofval <- gofstat(list(a_norm,a_logis),fitnames=c("norm","logis"))
gofval

In [ ]:

pv.chi.norm <- gofval$chisqpvalue["norm"]
pv.chi.logis <- gofval$chisqpvalue["logis"]
pv.cvm <- nortest::cvm.test(precio)$p.value  #Solamente para Normal 
pv.ad <-  nortest::ad.test(precio)$p.value   #Solamente para Normal 
pv.ks <- nortest::lillie.test(precio)$p.value  #Solamente para Normal 
pv.pea <-  nortest::pearson.test(precio)$p.value   #Solamente para Normal 

pvalue_norm <- data.frame(ChiC=pv.chi.norm,CvM=pv.cvm,AD=pv.ad,KS=pv.ks,PEA=pv.pea)
if (!is.na(pv.chi.logis))
{ pvalue_norm <- rbind(pvalue_norm,data.frame(ChiC=pv.chi.logis,CvM=NaN,AD=NaN,KS=NaN,PEA=NaN))
} else {pvalue_norm <- rbind(pvalue_norm,data.frame(ChiC=NaN,CvM=NaN,AD=NaN,KS=NaN,PEA=NaN))}

row.names(pvalue_norm) <- c("norm","logis")
pvalue_norm

### Tal como vimos anteriormente, el valor-p puede no ser un buen indicador cuando hay muchos datos.

### Simulemos las distribuciones y comparémoslas con la distribución empírica.

In [ ]:
mu <- a_norm$estimate[1]
sigma <- a_norm$estimate[2]

loc <- a_logis$estimate[1]
scale <- a_logis$estimate[2]

In [ ]:
Z.norm <- rnorm(length(precio),mu,sigma)
Z.logis <- rlogis(length(precio),loc,scale)
X <- precio
leg <- c("Datos","Normal", "Logistica")
color <- c(1,2,4)
eX <- ecdf(X)
eZN <- ecdf(Z.norm)
eZL <- ecdf(Z.logis)

plot(eX, ylab="Fn(x)", main="Distribución empirica",col=color[1], verticals = TRUE,do.points=FALSE,xlim=c(min(X),max(X)))
legend(0.75*max(X),0.1, leg[1], col=color[1], lty=1)

par(new=T)
plot(eZN, ylab="Fn(x)", main="Distribución empirica",col=color[2],verticals = TRUE,do.points=FALSE,xlim=c(min(X),max(X)))
legend(0.75*max(X),0.2, leg[2],  col=color[2], lty=1)

par(new=T)
plot(eZL, ylab="Fn(x)", main="Distribución empirica",col=color[3],verticals = TRUE,do.points=FALSE,xlim=c(min(X),max(X)))
legend(0.75*max(X),0.3, leg[3],  col=color[3], lty=1)



### Ambas son razonables. Quedémonos con la Normal.

## Ahora que tenemos una distribución, hagamos un pronóstico para el precio.

### Por ejemplo, un rango de valores que contiene el precio con probabilidad 95% es dado por

### $\mu \pm 1.96\sigma$, donde $\mu$ y $\sigma$ son los parámetros de la Normal ajustada

In [ ]:
sprintf("Pronóstico del precio (95%%): [%5.2f, %5.2f]",mu-1.96*sigma,mu+1.96*sigma )

### ¿Qué problemas hay con ese pronóstico?

### ¿Cómo se puede mejorarlo?

### Veamos como varian los precios por mes. Para ello, vamos a utilizar la librería "lubridate", con la cual queda mucho más fácil lidiar con la variable "time"





In [ ]:
install.packages('lubridate')
library(lubridate)


### Aqui vemos como es útil la estructura de data.table! En este ejemplo:

####   energy[ cuales filas,  qué hacer con cada columna seleccionada,    como agrupar las filas]

In [ ]:
precio_mes <- energy[,.(preciomedio=mean(`price actual`)),by=list(mes=month(time))]
precio_mes

In [ ]:
precio_mes <- cbind(precio_mes,month.abb[precio_mes$mes]) #Agrega columna con nombre de mes
names(precio_mes)[3] <- "mes_abrv"
plot.mes <- ggplot(precio_mes, aes(reorder(mes_abrv,mes),preciomedio))+
            geom_point(size=3,na.rm = TRUE)+
            labs(x = 'Mes', y = 'Precio')+
            mytheme
plot.mes

### Variación por hora

In [ ]:
precio_hora <- energy[,.(preciomedio=mean(`price actual`)),by=list(hora=hour(time))]
#head(precio_hora)

plot.hora <- ggplot(precio_hora, aes(hora,preciomedio))+
            geom_point(size=3,na.rm = TRUE)+
            labs(x = 'Hora', y = 'Precio')+
            mytheme
plot.hora



### Variación por día, fijando un mes (julio)

In [ ]:
precio_dia <- energy[month(time)==7,
                  .(preciomedio=mean(`price actual`)),by=list(dia=mday(time))]
head(precio_dia)

plot.dia <- ggplot(precio_dia, aes(dia,preciomedio))+
            geom_point(size=3, na.rm = TRUE)+
            labs(x = 'Día', y = 'Precio')+
            mytheme

plot.dia

### Vemos que el precio depende bastante del mes y de la hora. ¿Qué otras dependencias hay?

### Veamos por ejemplo los datos de  demanda real (load)

In [ ]:
dem_hora <- energy[,.(demmedia=mean(`total load actual`)),by=list(hora=hour(time))]
head(dem_hora)

### Tenemos que limpiar los datos! 

In [ ]:
dem_data <- energy[,.(`total load actual`),by=list(hora=hour(time))]  #sin cálculo de media
dem_data[is.na(dem_data$`total load actual`)]   #Elementos problemáticos


### Eliminamos los NA

In [ ]:
dem_data <- dem_data[!is.na(dem_data$`total load actual`)]
dem_hora <- dem_data[,.(demmedia=mean(`total load actual`)),by=hora]
head(dem_hora)

In [ ]:
plot.dem.hora <- ggplot(dem_hora, aes(hora,demmedia))+
            geom_point(size=3,na.rm = TRUE)+
            labs(x = 'Hora', y = 'Demanda')

plot.dem.hora+
    geom_line(na.rm = TRUE)+
    mytheme
                
plot.hora+
    geom_line(na.rm = TRUE, color="red")+
    mytheme
 


## Qué concluimos del análisis?

### Claramente, el precio depende del mes, día y demanda 

## ¿Cómo podemos usar esa información para hacer un mejor pronóstico del precio?

### Regresión!

### Podemos además incluir otros factores - pero siempre cuando utilicemos apenas los valores *pronosticados* de la base de datos, no los valores actuales.

### Asumamos que queremos hacer el pronóstico para el 31/7/2018, entonces separamos los datos de ese día del análisis para usarlos como test

In [ ]:
data7<-energy[year(time)<=2018 &
                month(time)==7 & mday(time)<31,
             .(anio=year(time),  
              dia= mday(time), 
               hora= hour(time),
                solar=`forecast solar day ahead`,
                eol=`forecast wind onshore day ahead`,
                dem=`total load forecast`,
                precio=`price actual`) ]

test7<-energy[year(time)==2018 &
                month(time)==7 & mday(time)==31,
             .(anio=year(time),  
              dia= mday(time), 
               hora= hour(time),
                solar=`forecast solar day ahead`,
                eol=`forecast wind onshore day ahead`,
                dem=`total load forecast`)]






In [ ]:
head(data7)
head(test7)

### Variable dependiente

In [ ]:
price7_test <- energy[year(time)==2018 &
                month(time)==7 & mday(time)==31,.(precio=`price actual`)]

head(price7_test)

## Corramos la regresión

### Chequeemos las correlaciones entre las variables

In [ ]:
pairs(data7)

### Modelo de regresión

In [ ]:
mod7 <- with(data7,lm(formula=precio ~ anio+hora+solar+eol+dem))
summary(mod7)

### Eliminemos las variables innecesarias

In [ ]:
mod7<-with(data7,lm(formula=precio ~ hora+eol+dem))
summary(mod7)

## En base a la regresión, hagamos un pronóstico de precios para cada hora del día 31/7/2018.

In [ ]:
pron_horario <- data.frame(hora=0:23, pronostico=predict(mod7,newdata=test7))
plot.pronostico <- ggplot(pron_horario,aes(hora,pronostico))+
                    geom_line(color="red")+
                    labs(x = 'Hora', y = 'Precio pronosticado',
                          title='Pronóstico para el día 31/7/2018')+
                    mytheme
plot.pronostico


### ¿Qué tan bueno es este pronóstico?

### ¿Qué falta es este pronóstico?





















### No hay ningún margen de error!

### ¿Cómo estimar los errores? 

## Veamos un caso más sencillo con apenas 1 variable (load forecast), tomando apenas los 3 primeros días

In [ ]:
data7red <- data7[1:72,.(dem,precio)]
mod_red <- with(data7red,lm(formula= precio ~ dem))
summary(mod_red)

In [ ]:
plot.dem_precio <- ggplot(data7red,aes(dem,precio))+
                        labs(x = 'Demanda pronosticada', y = 'Precio')+
                        geom_point()+
                        geom_smooth(method="lm", se=FALSE)+
                        mytheme
plot.dem_precio



### ¿Cómo estimar el error de la regresión para un dato específico de demanda pronosticada? 

### La clave es mirar los **residuos**. Recordemos que la ecuación de regresión es  $ Y = a + bX + \epsilon$,  donde $\epsilon$ es el residuo

In [ ]:
plot(data7red$dem,data7red$precio,col=4,  ylab="precio",xlab="demanda pronosticada")
lines(data7red$dem,fitted(mod_red))
segments(data7red$dem,fitted(mod_red),data7red$dem,data7red$precio)

### O sea, para un valor **fijo** de $X=X_0$ (en el ejemplo, demanda pronosticada), la variable $Y$ se distribuye (aproximadamente) de la **misma forma que los residuos.**

### Veamos entonces como se distribuyen los residuos:

In [ ]:
res<-resid(mod_red)
hist(res)

In [ ]:
descdist(res)

In [ ]:
plot.legend<-c("Normal", "Uniforme")
a_norm<-fitdist(res, "norm")
a_unif<-fitdist(res, "unif")

denscomp(list(a_norm,a_unif),fitcol=c(2,4), legendtext=plot.legend)

In [ ]:
gofval <- gofstat(list(a_norm,a_unif),fitnames=c("norm","unif"))
gofval

In [ ]:
pv.chi.norm <- gofval$chisqpvalue["norm"]
pv.chi.unif <- gofval$chisqpvalue["unif"]
pv.cvm <- nortest::cvm.test(res)$p.value  #Solamente para Normal 
pv.ad <-  nortest::ad.test(res)$p.value   #Solamente para Normal 
pv.ks <- nortest::lillie.test(res)$p.value  #Solamente para Normal 
pv.pea <-  nortest::pearson.test(res)$p.value   #Solamente para Normal 

pvalue_norm <- data.frame(ChiC=pv.chi.norm,CvM=pv.cvm,AD=pv.ad,KS=pv.ks,PEA=pv.pea)
pvalue_norm <- rbind(pvalue_norm,data.frame(ChiC=pv.chi.unif,CvM=NaN,AD=NaN,KS=NaN,PEA=NaN))
pvalue_norm

### Vemos que la distribución Normal provee un ajuste aceptable a los residuos.

### O sea, para un valor **fijo** de $X=X_0$ (en el ejemplo, demanda pronosticada), la variable $Y$ tiene(aproximadamente) distribucion Normal con

### media = $a + bX_0$,  y desviación estandar=D.E. de los residuos.

## Volvamos al caso original (precio vs. demanda, hora, etc.)


### Analicemos los residuos

In [ ]:
res7<-resid(mod7)
hist(res7)

In [ ]:
descdist(res7)

In [ ]:
a_norm<-fitdist(res7, "norm")
denscomp(a_norm,fitcol=2, legendtext=plot.legend)

In [ ]:
ppcomp(a_norm)

In [ ]:
qqcomp(a_norm)

### Utilicemos la D.E. de los residuos para calcular el error del pronóstico; el argumento int="p" de la función *predict* hace justamente eso

In [ ]:
pron_horario <- predict(mod7,newdata=test7,level=0.95,int="p")   #Añade intervalo de predicción
#head(fore_hour)


#Arma el dataframe con los datos pronosticados
types <- c("precio"="real", "fit"="pronóstico","lwr"="inf","upr"="sup")
horas <- data.frame(hora=0:23)
pron.df <- cbind(horas,pron_horario)
#head(pron.df)
pron <- pron.df %>% tidyr::gather("type", "precios", 2:4) 
#head(pron)
plot.pronostico <- ggplot(pron,aes(hora,precios,group=type,color=types[type]))+
                     geom_line()+
                     labs(x = 'Hora', y = 'Precios',
                          title='Pronóstico 95% para el día 31/7/2018',
                          color="Precios:")+
                      mytheme
                 
plot.pronostico

### ¿Cómo se compara con el  pronóstico anterior que habíamos hecho sin tomar en cuenta la información adicional?

In [ ]:
sprintf("Pronóstico del precio (95 %%): [%5.2f, %5.2f]",mu-1.96*sigma,mu+1.96*sigma )

### ¿Cómo se compara con el  pronóstico con los valores reales observados?

In [ ]:

real.df <- cbind(horas,price7_test)
#head(real.df)
real <- real.df %>% tidyr::gather("type", "precios", 2) 
#head(real)
                 
plot.pronostico+geom_line(data=real,aes(hora,precios,color=types[type]))




